# COGS 185 Final Project: CRF (3, 2.5k, 2.5k)

## Step 1: Installations and Imports

In [1]:
!pip install dlib
!pip install sklearn-crfsuite

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import warnings; warnings.simplefilter('ignore')
import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
oe = preprocessing.OneHotEncoder(sparse=False)
import dlib
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline
import timeit
label_encoder = preprocessing.LabelEncoder()
import pickle
import random
import sklearn_crfsuite
from sklearn.preprocessing import LabelEncoder
from sklearn_crfsuite import CRF, metrics

## Step 2: Function Definitions

In [3]:
# Read the entire dataset into lists or list of lists
def read_OCR(filename, n_features):
    F = open(filename)
    dataset = {}
    dataset['ids'] = []#np.zeros(n_examples, dtype=int)
    dataset['labels'] = []#np.zeros(n_examples,dtype=int)
    dataset['labelDic'] = {} # To profile the distribution of labels
    dataset['next_ids'] = []#np.zeros(n_examples,dtype=int)
    dataset['word_ids'] = []#np.zeros(n_examples,dtype=int)
    dataset['positions'] = []#np.zeros(n_examples,dtype=int)
    dataset['folds'] = []#np.zeros(n_examples,dtype=int)
    dataset['features'] = []#np.zeros([n_examples,n_features])
    
    for str_line in F.readlines():
        #line0 = map(iors, filter(None, re.split('\t', str_line.strip())))
        ## ATTENTION: If you are using Python3, use the following line instead
        line0 = list(map(iors, filter(None, re.split('\t', str_line.strip()))))


        dataset['ids'].append(int(line0.pop(0)))
        dataset['labels'].append(l2i(line0.pop(0))) # The label is converted into integer('a'=>0, 'z'=>25)
        if dataset['labels'][-1] in dataset['labelDic']:
            dataset['labelDic'][dataset['labels'][-1]] += 1
        else:
            dataset['labelDic'][dataset['labels'][-1]] = 1
            
        dataset['next_ids'].append(int(line0.pop(0)))
        dataset['word_ids'].append(int(line0.pop(0)))
        dataset['positions'].append(int(line0.pop(0)))
        dataset['folds'].append(int(line0.pop(0)))
        if len(line0) != 128:  # Sanity check of the length
            print (len(line0))
        dataset['features'].append(line0)

    return dataset

In [4]:
# Extract the first L letters in a word

def structurize1(dataset, N, L):
    d_features = len(dataset['features'][0])
    y = dataset['labels']
    X = dataset['features']
    next_id = dataset['next_ids']

    labels = np.zeros((N, L))
    features = np.zeros((N, L*d_features))
    
    # Extract only one structured example
    def extract(iN, loc):
        labels[iN] = y[loc:loc+L]
        features[iN] = np.array(X[loc:loc+L]).ravel().tolist()
        iN += 1
        return iN
    
    iN = 0
    iN = extract(iN, 0)
    
    for key, value in enumerate(y):
        if next_id[key] == -1:
            iN = extract(iN, key+1)
            
            if iN == N:
                break
    
    c = list(zip(labels, features))
    random.shuffle(c)
    labels, features = zip(*c)
    
    return np.array(labels), np.array(features)      

In [5]:
# Convert features to a format suitable for CRF
def convert_features(features, L, d):
    sequences = []
    for i in range(len(features)):
        sequence = []
        for j in range(L):
            feature_dict = {}
            for k in range(d):
                feature_dict[f'f{k}'] = features[i][j * d + k]
            sequence.append(feature_dict)
        sequences.append(sequence)
    return sequences

# Convert the labels to a format suitable for CRF
def convert_labels(labels, L):
    sequences = []
    for i in range(len(labels)):
        sequence = []
        for j in range(L):
            sequence.append(str(labels[i][j]))
        sequences.append(sequence)
    return sequences

In [6]:
# Windows Length
L = 3
# Number of examples
N = 5000
# Length of a feature
d = 128
# The hyper-parameter for icm search
Niter = 2               

In [7]:
def l2i(a):
        return int(ord(a)-ord('a'))
def i2l(i):
    if i >= 0:
        return chr(i+ord('a'))
    else:
        return '_'
def iors(s):
    try:
        return int(s)
    except ValueError: # if it is a string, return a string
        return s

In [8]:
dataset1 = read_OCR('OCRdataset/letter.data', d)

# Step 3: Concatenating and Structurizing

1) Find 5000 words, split them into 2500 for training, and 2500 for testing.

2) Chop first 3 characters from each word.

3) Now there are 2500 two-words pairs for training, and 2500 for testing.

4) Construct new structures data based on this pairs.
- Ex: 
    - apple => ap
    - banana => ba

In [9]:
labels1, features1 = structurize1(dataset1, N, L)

# Step 4: Convert Features and Labels for CRF

In [10]:
features_crf = convert_features(features1, L, d)
labels_crf = convert_labels(labels1, L)

# Step 5: Split Data into Training and Testing Sets

In [11]:
split = int(0.5 * N)
X_train, X_test = features_crf[:split], features_crf[split:]
y_train, y_test = labels_crf[:split], labels_crf[split:]

# Step 6: Create, Train, and Evaluate CRF Model

In [12]:
#Create and train the model
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

In [13]:
#Evaluate the model
y_pred_train = crf.predict(X_train)
y_pred_test = crf.predict(X_test)

train_accuracy = metrics.flat_f1_score(y_train, y_pred_train, average='weighted')
test_accuracy = metrics.flat_f1_score(y_test, y_pred_test, average='weighted')

print("Training accuracy (F1 score):", train_accuracy)
print("Test accuracy (F1 score):", test_accuracy)

Training accuracy (F1 score): 0.98612832440661
Test accuracy (F1 score): 0.8634722399343902


In [14]:
# Save the model
with open('crf_weights3_2.5_2.5.pkl', 'wb') as f:
    pickle.dump(crf, f)

# Load the model
with open('crf_weights3_2.5_2.5.pkl', 'rb') as f:
    crf_loaded = pickle.load(f)

In [15]:
# Evaluate the loaded model
y_pred_test_loaded = crf_loaded.predict(X_test)
loaded_test_accuracy = metrics.flat_f1_score(y_test, y_pred_test_loaded, average='weighted')
print("Loaded model test accuracy (F1 score):", loaded_test_accuracy)

Loaded model test accuracy (F1 score): 0.8634722399343902
